<a href="https://colab.research.google.com/github/chacha86/pythonai2/blob/main/%EB%89%B4%EC%8A%A4_%ED%86%B5%EA%B3%84_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%99%9C%EC%9A%A9%ED%95%9C_%EB%AC%B8%EC%A0%9C_%ED%92%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


## 뉴스데이터와 통계 데이터 로드

kbs_news = pd.read_json('/content/KBS_news.json')
mbc_news = pd.read_json('/content/MBC_news.json')
sbs_news = pd.read_json('/content/SBS_news.json')

kbs_stat = pd.read_json('/content/KBS_stat.json')
mbc_stat = pd.read_json('/content/MBC_stat.json')
sbs_stat = pd.read_json('/content/sbs_stat.json')

kbs_cnt = pd.read_json('/content/KBS_replCnt.json')
mbc_cnt = pd.read_json('/content/MBC_replCnt.json')
sbs_cnt = pd.read_json('/content/SBS_replCnt.json')

In [ ]:
## 각 언론사별 데이터 합치기

all_news = pd.concat([kbs_news, mbc_news, sbs_news])
all_stat = pd.concat([kbs_stat, mbc_stat, sbs_stat])
all_cnt = pd.concat([kbs_cnt, mbc_cnt, sbs_cnt])

In [ ]:
## 뉴스 데이터와 통계데이터 merge하기(뉴스 데이터와 통계 데이터는 구조가 다르므로 concat이 아닌 merge를 사용)
news_stat = pd.merge(all_news, all_stat, how='left', on='id')
result = pd.merge(news_stat, all_cnt, how='left', on='id')

In [ ]:
# 수집한 뉴스데이터를 이용해 다음 문제를 풀어주세요.
## 1. 20대가 가장 많이 본(댓글을 많이 작성한) 뉴스
### 결측치 -> 제거
result2 = result.dropna()

### 문자를 숫자로 변환하기 위해 ,를 제거
def remove_comma(s) :
  s = str(s)
  if len(s) > 3 :
    s = s.replace(',', '')

  return s

## apply를 이용해 댓글수의 , 제거
result2['count2'] = result2['count'].apply(remove_comma).astype('int')

In [ ]:
## 20대 비율을 수치로 변환
result2['20대 비율'] = result2['20대'].str.replace('%', '').astype('float') * 0.01

In [ ]:
## 댓글수와 20대 비율로 댓글 수치 계산
result2[['20대 비율', 'count2']]
result2['20대 댓글수'] = result2['20대 비율'] * result2['count2']

In [ ]:
## 정렬을 이용해 20대 댓글 수가 가장 높은 뉴스 찾기
result2['20대 댓글수'].sort_values(ascending=False)

result2.loc[22]['link']

'https://n.news.naver.com/article/214/0001298229?ntype=RANKING'

In [ ]:
## 숫자, 문자 -> 다 문자로 형변환 -> replace 함수 적용

## 문자인 댓글 수를 수치로 변환하기 위한 방법2. 모두 문자로 바꾼 후 replace로 , 지움. 이후 astype으로 형변환
result2['count2'] = result2['count'].astype('str').str.replace(',','').astype('int')

In [ ]:

## 연령별 성별별 댓글수를 구하기 위해 문자를 수치로 변환하고 댓글 수 구하기
def remove_percent(val) :
  return val.replace('%', '')

tmp = result2[['남자', '여자', '10대', '20대', '30대',
       '40대', '50대', '60대']].applymap(remove_percent).astype('int') * 0.01


tmp['댓글수'] = result2['count2']

def calculate_reply(row):
  return row.drop('댓글수') * row['댓글수']

tmp.apply(calculate_reply, axis=1)

result2.loc[:, ['남자', '여자', '10대', '20대', '30대', '40대', '50대', '60대', 'count2']] = tmp.apply(calculate_reply, axis=1)

In [ ]:
result2['count2'] = tmp['댓글수']

<ipython-input-132-805c7000705b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result2['count2'] = tmp['댓글수']


In [ ]:
## 2. 가장 댓글이 적은 뉴스의 댓글 수와 뉴스 링크, 뉴스 번호

result2.loc[result2['count2'].sort_values().head(1).index, ['link', 'id']] # 댓글 수로 오름차순 정렬한 후 가장 위에 것 조회



,link,id
50,https://n.news.naver.com/article/055/000108774...,1087745


In [ ]:
## 3. 언론사 번호와 언론사 이름으로 구성된 데이터 프레임을 만들고 merge를 이용해 각 뉴스에 언론사 이름을 붙여주기
## kbs 언론사
kbs_news['언론사'] = 'KBS'


result3 = pd.merge(result2, kbs_news[['id','언론사']], how='left', on = 'id') # result2 에 kbs 언론사 정보를 merge. kbs를 제외한 나머지 언론사는 NaN으로 채워질 것임.
result3['언론사']

mbc_news['언론사'] = 'MBC'

idx = pd.merge(result3, mbc_news['id'], how='right', on='id')['link'].notna() # result3과 mbc 언론사 정보를 merge. mbc 뉴스를 기준으로 outer merge했으므로 result3쪽에 NaN이 생길 것. NaN값을 제외한 mbc뉴스만 타겟을 삼기 위해 idx를 뽑아옴
target = pd.merge(result3, mbc_news['id'], how='right', on='id').loc[idx]['id'] # 위에서 얻은 idx로 행을 찾은 후 id값을 얻어옴.

result4 = result3.set_index('id') # 해당 id를 index로 사용.
result4.loc[target, '언론사'] = 'MBC' # mbc 뉴스만 골라낼 수 있음. mbc 뉴스에만 언론사 컬럼 값을 'MBC'로 대입



In [ ]:
result4.loc[result4['언론사'].isna(), '언론사'] = 'SBS' # kbs, mbc를 제외한 나머지 값은 명백히 sbs이므로 'SBS' 대입.

In [ ]:
## 4. 각 언론사별 평균 댓글 수(댓글 수로 내림차순 정렬)
result4.groupby('언론사')['count2'].mean().sort_values(ascending=False) # 내림차순 정렬해서 풀이


In [ ]:

## 5. 여성의 댓글수가 가장 많은 언론사
result4.groupby('언론사')['여성'].sum().sort_values(ascending=False).head(1) # 내림차순 정렬해서 풀이

In [ ]:
## 6. 각 언론사 별 댓글을 많이 작성한 연령대 top3 -> wide -> long

reulst5 = pd.melt(result4, id_vars = ['언론사'], value_vars=['10대', '20대', '30대', '40대', '50대', '60대'], var_name='연령대', value_name='댓글수')
pd.DataFrame(reulst5.groupby(['언론사', '연령대'])['댓글수'].sum()).reset_index().sort_values('댓글수', ascending=False).groupby('언론사').head(3)